# ctypes로 Python에서 C++ 클래스 호출하기

* 원출처
  - https://www.auctoris.co.uk/2017/04/29/calling-c-classes-from-python-with-ctypes/#comments
* 2차 번역
  - https://sdr1982.tistory.com/212

저는 파이썬에서 C++ 클래스를 호출하고 싶어서 최근에 스스로 방법을 찾았습니다. 저는 (Thrift를 사용하여 전에 했던 것처럼 - [Python과 C++을 위한 Apache Thrift 사용하기](https://www.auctoris.co.uk/2016/08/17/using-apache-thrift-for-python-c/)를 보세요.) 분리된 프로세스 를 호출하고 싶지 않았고 C++ 라이브러리를 직접 호출하고 싶었습니다.

저는 진행하기 전에 이를 Java로 하기 위한 다양한 방법이 있다는 것을 말하고 싶습니다. 그리고 저는 작동한 것 중 하나를 선택하였습니다. 다른 기술도 사용 가능하며 어떤 기술이 '최상'인지에 대한 의견은 상당히 분분해 보입니다.

C++ 클래스로 시작하기 위해 평범하게 작성하였습니다.

In [ ]:
%%writefile foo.cpp
#include <iostream>

// A simple class with a constuctor and some methods...
// 생성자와 몇 개의 메소드를 가지는 간단한 클래스...

class Foo
{
    public:
        Foo(int);
        void bar();
        int foobar(int);
    private:
        int val;
};

Foo::Foo(int n)
{
    val = n;
}

void Foo::bar()
{
    std::cout << "Value is " << val << std::endl;
}

int Foo::foobar(int n)
{
    return val + n;
}

Writing foo.cpp


다음 ctypes 시스템은 C++을 사용할 수 없기 때문에 C++ 코드 주변에 C wrapper를 놓을 것입니다. 이를 하기 위해 파일 제일 밑에 다음 부분에 코드를 추가합니다.

In [ ]:
add_src = """

extern "C"
{
    Foo* Foo_new(int n) {return new Foo(n);}
    void Foo_bar(Foo* foo) {foo->bar();}
    int Foo_foobar(Foo* foo, int n) {return foo->foobar(n);}
}"""

f = open('foo.cpp', 'a')
f.write(add_src)
f.close()

호출하기 원하는 각 메소드를 클래스 기반이 아닌 이름으로 제공해야 함을 알아두세요.

우리는 우리 코드에서 libfoo.so 파일을 빌드해야 합니다.

다음을 쉘에서 입력하세요.

In [ ]:
!g++ -c -fPIC foo.cpp -o foo.o

In [ ]:
!g++ -shared -Wl,-soname,libfoo.so -o libfoo.so foo.o

In [ ]:
!ls -lrt

total 32
drwxr-xr-x 1 root root  4096 May  6 13:44 sample_data
-rw-r--r-- 1 root root   586 May 20 08:43 foo.cpp
-rw-r--r-- 1 root root  4424 May 20 08:44 foo.o
-rwxr-xr-x 1 root root 13320 May 20 08:44 libfoo.so


In [ ]:
!cat foo.cpp

#include <iostream>

// A simple class with a constuctor and some methods...
// 생성자와 몇 개의 메소드를 가지는 간단한 클래스...

class Foo
{
    public:
        Foo(int);
        void bar();
        int foobar(int);
    private:
        int val;
};

Foo::Foo(int n)
{
    val = n;
}

void Foo::bar()
{
    std::cout << "Value is " << val << std::endl;
}

int Foo::foobar(int n)
{
    return val + n;
}

extern "C"
{
    Foo* Foo_new(int n) {return new Foo(n);}
    void Foo_bar(Foo* foo) {foo->bar();}
    int Foo_foobar(Foo* foo, int n) {return foo->foobar(n);}
}

또는 CMake를 사용할 수 있습니다.

다음은 foo.cpp를 빌드하기 위한 CMakeLists.txt 입니다.

```
cmake_minimum_required(VERSION 2.8.9)
set(CMAKE_CXX_FLAGS "${CMAKE_CXX_FLAGS} -std=c++11 -Wall")
set(CMAKE_RUNTIME_OUTPUT_DIRECTORY "${CMAKE_SOURCE_DIR}")
set(CMAKE_MACOSX_RPATH 1)

project (foo)
set (SOURCE foo.cpp)
add_library(foo MODULE ${SOURCE}) 
```

저는 Mac에서 빌드를 해서 MacOS를 위해 4번째 줄을 추가하였습니다. 리눅스에서도 잘 작동하겠지만 필요는 없습니다.

이제 C++로 컴파일 된 내용을 작성합니다. 우리는 클래스에 대한 Python wrapper를 빌드하려 합니다.

In [ ]:
%%writefile foo.py
import ctypes

lib = ctypes.cdll.LoadLibrary('./libfoo.so')

class Foo(object):
    def __init__(self, val):
        lib.Foo_new.argtypes = [ctypes.c_int]
        lib.Foo_new.restype = ctypes.c_void_p

        lib.Foo_bar.argtypes = [ctypes.c_void_p]
        lib.Foo_bar.restype = ctypes.c_void_p

        lib.Foo_foobar.argtypes = [ctypes.c_void_p, ctypes.c_int]
        lib.Foo_foobar.restype = ctypes.c_int

        self.obj = lib.Foo_new(val)
        
    def bar(self):
        lib.Foo_bar(self.obj)
   
    def foobar(self, val):
        return lib.Foo_foobar(self.obj, val)

Writing foo.py


리턴 값의 타입과 argument 타입을 정의하는 요구사항을 적으세요. (하나도 리턴하지 않으면 예시로 void를 리턴합니다.) 이것이 없으면 segmentation fault(등)를 얻을 것입니다.

이제 모든 것을 다 하였고 모듈을 빌드해야 합니다. 파이썬에서 간단히 그것을 import할 수 있습니다.

예를 들어

```python
from foo import Foo
# 우리는 5라는 값으로 Foo 객체를 생성할 것입니다...
f=Foo(5)
# f.bar()를 호출하는 것은 값을 포함한 메시지를 출력할 것입니다...
f.bar()
# 이제 f, Foo 객체에서 저장되어 있는 값에 값(7)을 더하기 위해 foobar를 사용합니다.
print (f.foobar(7))
# 또 한 번 같은 메소드를 호출합니다 - 이 번엔 일반적인 파이썬 정수를 
# 보여줄 것입니다...
x = f.foobar(2)
print (type(x))
```

In [ ]:
!python

Python 3.7.10 (default, May  3 2021, 02:48:31) 
[GCC 7.5.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> from foo import Foo
>>> f=Foo(5)
>>> f.bar()
Value is 5
>>> print (f.foobar(7))
12
>>> x = f.foobar(2)
>>> x
7
>>> print (type(x))
<class 'int'>
>>> quit()


이 간단한 데모를 위한 전체 소스 코드는 여기에 있습니다.

https://github.com/Auctoris/ctypes_demo

# JNA로 Java에서 C++ 클래스 호출하기

## 사전작업

* java 설치(Java 8 다운그레이드)

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:13 https://developer.

* Java 환경변수 설정

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = "/usr/lib/jvm/java-8-openjdk-amd64/bin:" + os.environ["PATH"]

* Java 버전 확인.
  - Java 8 버전인지 확인.
  - 설치할 Gradle과 호환성을 맞추기 위해 Java 다운그레이드

In [ ]:
!java -version

openjdk version "1.8.0_292"
OpenJDK Runtime Environment (build 1.8.0_292-8u292-b10-0ubuntu1~18.04-b10)
OpenJDK 64-Bit Server VM (build 25.292-b10, mixed mode)


* gradle 설치
  - https://yallalabs.com/devops/how-to-install-gradle-ubuntu-18-04-ubuntu-16-04/

* Gradle 다운로드
  - 3.4.1 버전의 **Gradle**을 받기 위해 다음처럼 진행.

In [ ]:
!wget https://services.gradle.org/distributions/gradle-3.4.1-bin.zip -P /tmp

--2021-05-20 08:55:53--  https://services.gradle.org/distributions/gradle-3.4.1-bin.zip
Resolving services.gradle.org (services.gradle.org)... 104.18.191.9, 104.18.190.9, 2606:4700::6812:be09, ...
Connecting to services.gradle.org (services.gradle.org)|104.18.191.9|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.gradle-dn.com/distributions/gradle-3.4.1-bin.zip [following]
--2021-05-20 08:55:53--  https://downloads.gradle-dn.com/distributions/gradle-3.4.1-bin.zip
Resolving downloads.gradle-dn.com (downloads.gradle-dn.com)... 104.18.165.99, 104.18.164.99, 2606:4700::6812:a563, ...
Connecting to downloads.gradle-dn.com (downloads.gradle-dn.com)|104.18.165.99|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 70310446 (67M) [application/zip]
Saving to: ‘/tmp/gradle-3.4.1-bin.zip’

gradle-3.4.1-bin.zi 100%[===================>]  67.05M   177MB/s    in 0.4s    

2021-05-20 08:55:53 (177 MB/s) - ‘/tmp/gradle-3

In [ ]:
!sudo unzip -d /opt/gradle /tmp/gradle-*.zip

Archive:  /tmp/gradle-3.4.1-bin.zip
   creating: /opt/gradle/gradle-3.4.1/
  inflating: /opt/gradle/gradle-3.4.1/LICENSE  
   creating: /opt/gradle/gradle-3.4.1/media/
  inflating: /opt/gradle/gradle-3.4.1/media/gradle-icon-16x16.png  
  inflating: /opt/gradle/gradle-3.4.1/media/gradle-icon-24x24.png  
  inflating: /opt/gradle/gradle-3.4.1/media/gradle.icns  
  inflating: /opt/gradle/gradle-3.4.1/media/gradle-icon-512x512.png  
  inflating: /opt/gradle/gradle-3.4.1/media/gradle-icon-32x32.png  
  inflating: /opt/gradle/gradle-3.4.1/media/gradle-icon-128x128.png  
  inflating: /opt/gradle/gradle-3.4.1/media/gradle-icon-64x64.png  
  inflating: /opt/gradle/gradle-3.4.1/media/gradle-icon-256x256.png  
  inflating: /opt/gradle/gradle-3.4.1/media/gradle-icon-48x48.png  
   creating: /opt/gradle/gradle-3.4.1/init.d/
  inflating: /opt/gradle/gradle-3.4.1/init.d/readme.txt  
  inflating: /opt/gradle/gradle-3.4.1/NOTICE  
  inflating: /opt/gradle/gradle-3.4.1/getting-started.html  
   creating:

In [ ]:
import os
os.environ["GRADLE_HOME"] = "/opt/gradle/gradle-3.4.1"
os.environ["PATH"] = "/opt/gradle/gradle-3.4.1/bin:" + os.environ["PATH"]

* gradle version 확인.

In [ ]:
!gradle -v


------------------------------------------------------------
Gradle 3.4.1
------------------------------------------------------------

Build time:   2017-03-03 19:45:41 UTC
Revision:     9eb76efdd3d034dc506c719dac2955efb5ff9a93

Groovy:       2.4.7
Ant:          Apache Ant(TM) version 1.9.6 compiled on June 29 2015
JVM:          1.8.0_292 (Private Build 25.292-b10)
OS:           Linux 5.4.109+ amd64



* 출처 및 응용
  - http://www.auctoris.co.uk/2017/04/29/calling-c-classes-from-python-with-ctypes/#comments
  - https://sdr1982.tistory.com/263

저는 Java에서 C++ 클래스를 호출하고 싶어서 최근에 스스로 방법을 찾았습니다. 저는 C++ 라이브러리를 직접 호출하고 싶었습니다.

저는 진행하기 전에 이를 Java로 하기 위한 다양한 방법이 있다는 것을 말하고 싶습니다. 그리고 저는 작동한 것 중 하나를 선택하였습니다. 다른 기술도 사용 가능하며 어떤 기술이 '최상'인지에 대한 의견은 상당히 분분해 보입니다.

C++ 클래스로 시작하기 위해 평범하게 작성하였습니다.

```c++
#include <iostream> 
// A simple class with a constuctor and some methods... 
// 생성자와 몇 개의 메소드를 가지는 간단한 클래스... 
class Foo { 
  public: 
    Foo(int); 
    void bar(); 
    int foobar(int); 
  private: 
  int val; 
}; 
Foo::Foo(int n) { 
  val = n; 
} 
void Foo::bar() { 
  std::cout << "Value is " << val << std::endl; 
} 
int Foo::foobar(int n) { 
  return val + n; 
}
```

JNA에서는 C++을 사용할 수 없기 때문에 C++ 코드 주변에 C wrapper를 놓을 것입니다. 이를 하기 위해 파일 제일 밑에 다음 부분에 코드를 추가합니다.

```c++
// ctypes는 C와만 대화할 수 있기 때문에 C++ 클래스를 위한 C 함수를 정의합니다. 
extern "C" { 
  Foo* Foo_new(int n) {  return new Foo(n);  } 
  void Foo_bar(Foo* foo) {  foo->bar();  } 
  int Foo_foobar(Foo* foo, int n) {  return foo->foobar(n);  } 
}
```

호출하기 원하는 각 메소드를 클래스 기반이 아닌 이름으로 제공해야 함을 알아두세요.

우리는 우리 코드에서 libfoo.so 파일을 빌드해야 합니다.

다음을 쉘에서 입력하세요.



```shell
$ g++ -c -fPIC foo.cpp -o foo.o 
$ g++ -shared -W1,-soname,libfoo.so -o libfoo.so foo.o
```



gradle에서 JNA를 fat-jar로 컴파일하기 위해 빌드 스크립트를 만들었습니다.

gradle 3.4.1에서 잘 작동함을 확인하였습니다.

* fat-jar : 실행에 필요한 모든 라이브러리 class 파일을 하나의 jar 파일로 만듬.(빌드)

In [ ]:
%%writefile build.gradle
// Java 프로그램을 위한 기능을 제공하는 플러그인
apply plugin: 'java'
// 
apply plugin: 'com.github.johnrengelman.shadow'

// gradle 스크립트에서 외부 라이브러리를 참
buildscript {
    // JCenter라는 저장소. Maven과 Gradle 등 각종 빌드 도구에서 사용할 수 있는 공개 저장소
    repositories {
        jcenter()
    }
    // 의존 라이브러리. fat-jar를 만들기 위해 shadowJar라는 라이브러리 사용.
    dependencies {
        classpath 'com.github.jengelman.gradle.plugins:shadow:2.0.1'
    }
}

// 저장소 설정
repositories {
 // Apache Maven 중앙 저장소를 이용하기 위한 것
 mavenCentral()
}

// 의존 라이브러리. 
dependencies {
  // 컴파일시 의존 라이브러리. JNA 라이브러리 4.1을 추가로 사용해야 함.
  compile 'net.java.dev.jna:jna:4.1.0'
  compile 'net.java.dev.jna:jna-platform:4.1.0'
}

// jar task가 실행될 때 마다 shadowJar가 실행되게 하려면, 아래처럼 jar에 finalizedBy를 달아주면 된다.
jar {
   finalizedBy shadowJar
   // jar 파일을 만드려면 manifest 파일 정보가 필요함.
   // main class는 hello 패키지에 Foo 클래스임.
   manifest {
      attributes 'Main-Class': 'hello.Foo',
                 'Implementation-Title': 'Foo jna Project',
                 'Implementation-Version': '1.0'
   }
}

Writing build.gradle


* gradle script 참고주소 
  - https://araikuma.tistory.com/463
  - https://notpeelbean.tistory.com/entry/gradle-buildscript-dependencies-%EC%99%80-dependencies-%EC%9D%98-%EC%B0%A8%EC%9D%B4
  - https://blog.leocat.kr/notes/2017/10/11/gradle-shadowjar-make-fat-jar

자바 소스는 src/main/java/hello/Foo.java 로 작성하였습니다.

In [ ]:
!mkdir -p src/main/java/hello

In [ ]:
%%writefile src/main/java/hello/Foo.java
package hello;

import com.sun.jna.Native;
import com.sun.jna.Platform;
import com.sun.jna.Library;
import com.sun.jna.Pointer;

public class Foo {
    public interface FooLib extends Library {
        Pointer Foo_new(int n);
        void Foo_bar(Pointer foo);
        int Foo_foobar(Pointer foo, int n);
    }

    private String sopath;
    private FooLib INSTANCE;
    private Pointer self;

    private void loadLibrary(int n) {
        INSTANCE = (FooLib)Native.loadLibrary(
            sopath, FooLib.class
        );
        self = INSTANCE.Foo_new(n);
    }

    public Foo(int n) {
        sopath = "libfoo.so";
        loadLibrary(n);
    }
    public Foo(String sopath, int n) {
        this.sopath = sopath;
        loadLibrary(n);
    }

    public void bar() {
        INSTANCE.Foo_bar(self);
    }

    public int foobar(int n) {
        return INSTANCE.Foo_foobar(self, n);
    }

    public static void main(String[] args) {
        String path = System.getProperty("user.dir") + "/libfoo.so";
        //String path = "/content/libfoo.so";
        System.out.println(path);
        Foo f = new Foo(path,5);

        f.bar();

        System.out.println("f.foobar(7) = " + f.foobar(7));

        int x = f.foobar(2);
        System.out.println("x = " + x);
    }
}

Writing src/main/java/hello/Foo.java


gradle로 Java 소스를 build 합니다.

In [ ]:
!gradle build

Starting a Gradle Daemon (subsequent builds will be faster)
> Starting Daemon> Starting Daemon > Connecting to Daemon> Loading> Loading > settings> Loading> Configuring> Configuring > 0/1 projects> Configuring > 0/1 projects > root project> Configuring > 0/1 projects > root project > Compiling /content/build.gradle into local build cache > Compiling build file '/content/build.gradle' to cross build script cache> Configuring > 0/1 projects > root project > Compiling /content/build.gradle into local build cache> Configuring > 0/1 projects > root project> Configuring > 0/1 projects > root project > Resolving dependencies ':classpath'Download https://jcenter.bintray.com/com/github/jengelman/gradle/plugins/shadow/2.0.1/shadow-2.0.1.pom
> Configuring > 0/1 projects > root project > Resolving dependencies ':classpath'
Download https://jcenter.bintray.com/org/codehaus/groovy/groovy-backports-compat23/2.4.4/groovy-backports-compat23-2.4.4.pom
> Configuring > 0/1 projects > root project > Resolv

빌드를 하면 build/libs 경로에 2개 파일이 생성됩니다.

* content.jar: 원래 내 프로젝트
* content-all.jar: JNA 라이브러리가 포함된 Fat-JAR 프로젝트

In [ ]:
!ls -lrt build/libs/*.jar

-rw-r--r-- 1 root root    1878 May 20 09:05 build/libs/content.jar
-rw-r--r-- 1 root root 1606286 May 20 09:05 build/libs/content-all.jar


다음은 실행 결과입니다.

In [ ]:
!java -jar build/libs/content-all.jar

/content/libfoo.so
Value is 5
f.foobar(7) = 12
x = 7


다음처럼 Makefile을 생성합니다.



In [ ]:
%%writefile Makefile
SRCS    = foo.cpp
OBJS    = foo.o

CFLAGS = $(CFLAG) -D_REENTRANT -D_THREAD_SAFE -D$(_OSTYPE_)
CPPFLAGS= $(CPPFLAG) -D_REENTRANT -D_THREAD_SAFE -D$(_OSTYPE_)

all : libfoo.so

libfoo.so :
	g++ -fPIC -c $(SRCS)
	g++ -shared -Wl,-soname,$@ -o $@ $(OBJS)
	gradle build

clean:
	rm -f *.o core *.out .*list *.ln *.so
	gradle clean

Writing Makefile


C++, Java 소스를 make로 한꺼번에 build 할 수 있습니다.



In [ ]:
!make clean && make

rm -f *.o core *.out .*list *.ln *.so
gradle clean
> Connecting to Daemon> Loading> Configuring > 1/1 projects:clean

BUILD SUCCESSFUL

Total time: 1.179 secs
g++ -fPIC -c foo.cpp
g++ -shared -Wl,-soname,libfoo.so -o libfoo.so foo.o
gradle build
> Connecting to Daemon> Building 0% > :compileJava:compileJava
:processResources NO-SOURCE
:classes
:jar
> Building 33% > :shadowJar:shadowJar
> Building 33%:assemble
:compileTestJava NO-SOURCE
:processTestResources NO-SOURCE
:testClasses UP-TO-DATE
:test NO-SOURCE
:check UP-TO-DATE
:build

BUILD SUCCESSFUL

Total time: 1.708 secs


In [ ]:
!java -jar build/libs/content-all.jar

/content/libfoo.so
Value is 5
f.foobar(7) = 12
x = 7


In [ ]:
!ls -lrt /content

total 52
drwxr-xr-x 1 root root  4096 May  6 13:44 sample_data
-rw-r--r-- 1 root root   586 May 19 08:28 foo.cpp
-rw-r--r-- 1 root root  4424 May 19 08:28 foo.o
-rwxr-xr-x 1 root root 13320 May 19 08:28 libfoo.so
-rw-r--r-- 1 root root   580 May 19 08:28 foo.py
drwxr-xr-x 2 root root  4096 May 19 08:29 __pycache__
-rw-r--r-- 1 root root  1375 May 19 08:33 build.gradle
drwxr-xr-x 3 root root  4096 May 19 08:33 src
drwxr-xr-x 5 root root  4096 May 19 08:33 build
